In [1]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/특허/bbb2.csv", encoding='cp949')
print(df.columns)

Index(['title', 'independence', 'summation', 'year'], dtype='object')


In [2]:
# !pip install googletrans==4.0.0-rc1

In [3]:
# import pandas as pd
# from googletrans import Translator

# # 번역기 초기화
# translator = Translator()

# # 예제 데이터프레임 (한국어 데이터)

# # 데이터프레임 전체를 번역하는 함수
# def translate_dataframe(df, src_lang='ko', dest_lang='en'):
#     return df.applymap(lambda x: translator.translate(x, src=src_lang, dest=dest_lang).text if isinstance(x, str) else x)

# # 한국어 → 영어 번역
# df_translated = translate_dataframe(df)

# # 결과 출력
# print(df_translated.head())


In [4]:
# import pandas as pd
# import asyncio
# from googletrans import Translator

# # 번역기 초기화
# translator = Translator()

# # 비동기 번역 함수
# async def translate_text(text, src='ko', dest='en'):
#     translation = await translator.translate(text, src=src, dest=dest)
#     return translation.text

# # 데이터프레임을 번역하는 함수
# async def translate_dataframe(df, src='ko', dest='en'):
#     tasks = [[translate_text(cell, src, dest) if isinstance(cell, str) else cell for cell in row] for row in df.values]
#     translated_values = await asyncio.gather(*[asyncio.gather(*row) for row in tasks])
#     return pd.DataFrame(translated_values, columns=df.columns)

# # 번역 실행
# df_translated = asyncio.run(translate_dataframe(df))

# # 결과 출력
# print(df_translated)


In [5]:
# import pandas as pd
# from googletrans import Translator

# translator = Translator()

# df['이름_영어'] = df['이름'].apply(lambda x: translator.translate(x, src='ko', dest='en').text)
# df['설명_영어'] = df['설명'].apply(lambda x: translator.translate(x, src='ko', dest='en').text)

# print(df)


In [6]:
!pip install bertopic[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [7]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [24]:
def preprocessing(text):
    if isinstance(text, float):
        text = str(text)
    text = text.lower()
    text = re.sub(r"[^a-zA-Z]", " ", text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['claim', 'non'])
    tokens = [item for item in tokens if item not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens]
    return ' '.join(lemmatized_tokens)

df['text'] = df['title'] + ' ' + df['independence'] + ' ' + df['summation']

# Apply preprocessing to the text column
df['text'] = df['text'].apply(lambda x: preprocessing(x))

# Remove empty documents
combined_docs =df['text'][df['text'].str.strip().astype(bool)].astype(str)

# Set up CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words='english')

# Create BERTopic model
topic_model = BERTopic(vectorizer_model=vectorizer_model,min_topic_size=30,n_gram_range=(1, 2), calculate_probabilities=True, verbose=True)

# Generate document embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(combined_docs.tolist(), show_progress_bar=True)

# Perform topic modeling
topics, probabilities = topic_model.fit_transform(combined_docs.tolist(), embeddings)

# Display topic information
print(topic_model.get_topic_info())

print('Number of topics for each document:', len(topics))
print('Topic number of the first document:', topics[0])

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

2025-02-06 11:09:31,279 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-06 11:09:38,016 - BERTopic - Dimensionality - Completed ✓
2025-02-06 11:09:38,022 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-06 11:09:38,700 - BERTopic - Cluster - Completed ✓
2025-02-06 11:09:38,712 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-06 11:09:44,495 - BERTopic - Representation - Completed ✓


    Topic  Count                                       Name  \
0      -1    775       -1_second_comprise_position_aircraft   
1       0   1910       0_flight_information_vehicle_control   
2       1    169          1_seat_position_passenger_support   
3       2    167          2_power_charge_battery_electrical   
4       3    138     3_second_structure_comprise_structural   
5       4    126               4_rotor_propeller_say_thrust   
6       5    109                5_gear_drive_lock_land gear   
7       6    109                     6_ice_heat_air_surface   
8       7    108                7_fuel_tank_valve_fuel tank   
9       8     97                    8_duct_air_inlet_engine   
10      9     95                  9_door_position_lock_open   
11     10     78                10_wing_tip_second_position   
12     11     62            11_hydraulic_piston_valve_fluid   
13     12     56  12_extinguish_agent_extinguish agent_foam   
14     13     48             13_air_engine_compressor_b

In [10]:
# # Visualize topics
# try:
#     fig = topic_model.visualize_topics()
#     fig.show()
# except ValueError as e:
#     print(f"Error visualizing topics: {e}")

# # Visualize individual topics
# try:
#     topic_info = topic_model.get_topic_info()
#     valid_topic_ids = topic_info[topic_info['Topic'] != -1]['Topic'].tolist()  # Extract valid topic IDs only
#     for topic_id in valid_topic_ids:
#         try:
#             fig = topic_model.visualize_distribution(probabilities[topic_id], min_probability=0.015)
#             fig.show()
#         except ValueError as e:
#             print(f"Error visualizing topic {topic_id}: {e}")
# except ValueError as e:
#     print(f"Error retrieving topic info: {e}")

In [25]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,775,-1_second_comprise_position_aircraft,"[second, comprise, position, aircraft, configu...",[preloaded torque shaft flight control driveli...
1,0,1910,0_flight_information_vehicle_control,"[flight, information, vehicle, control, unman,...",[flight control device method flight control s...
2,1,169,1_seat_position_passenger_support,"[seat, position, passenger, support, assembly,...",[cradle recline mechanism fix shell aircraft s...
3,2,167,2_power_charge_battery_electrical,"[power, charge, battery, electrical, unman, ve...",[wireless power transmission system wireless p...
4,3,138,3_second_structure_comprise_structural,"[second, structure, comprise, structural, surf...",[assembly method connect aircraft wing aft spa...
5,4,126,4_rotor_propeller_say_thrust,"[rotor, propeller, say, thrust, second, axis, ...",[aircraft rotor wing conversion capabilities a...
6,5,109,5_gear_drive_lock_land gear,"[gear, drive, lock, land gear, land, shaft, wh...",[drive system aircraft land gear drive system ...
7,6,109,6_ice_heat_air_surface,"[ice, heat, air, surface, temperature, aircraf...",[de ice system airfoil aircraft system de ice ...
8,7,108,7_fuel_tank_valve_fuel tank,"[fuel, tank, valve, fuel tank, flow, pressure,...",[aircraft fuel system aircraft fuel tank syste...
9,8,97,8_duct_air_inlet_engine,"[duct, air, inlet, engine, intake, fan, open, ...",[airflow control system methods thereof airflo...


In [26]:
display(topic_model.get_topic(3))

[('second', 0.03126401854955961),
 ('structure', 0.027481055080464997),
 ('comprise', 0.026525918456168945),
 ('structural', 0.025794802476505167),
 ('surface', 0.02237929426178431),
 ('member', 0.021843849643164787),
 ('portion', 0.021782707308761477),
 ('fastener', 0.01934938708177985),
 ('end', 0.018266555319524612),
 ('joint', 0.01789678857606881)]

In [27]:
topic_model.visualize_topics(top_n_topics=32)

In [28]:
# 앞서 calculate_probabilities를 했다면 특정 Document에 대해서 Topic별 Probabilites를 확인할 수 있다.여기서 예시는 index 200번째 Document
topic_model.calculate_probabilities=True
topic_model.visualize_distribution(probabilities[1], min_probability=0.0015)

In [29]:
# Topic 별 Hierarchy를 보여준다.
topic_model.visualize_hierarchy(top_n_topics=50)

In [30]:
# Topic을 대표하는 상위 단어 5개씩 보여준다.
topic_model.visualize_barchart(top_n_topics=50)

In [31]:
# Topic간 유사도를 Cosine Similarity로 계산후 Heatmap으로 표현
topic_model.visualize_heatmap(n_clusters=2, width=1000, height=1000)

In [32]:
# Topic내 대표하는 단어들에 대해서 c-tf-idf로 계산해서 각 단어가 Topic에서 차지하는 중요도를 계산했던 것을 Rank 순대로 보여준다.
topic_model.visualize_term_rank()

In [33]:
timestamps1 = df['year'].tolist()

In [34]:
ll = df[df['year'].isnull()].index
print(ll)

Index([], dtype='int64')


In [35]:
timestamps1 = pd.Series(timestamps1).dropna().tolist()

In [36]:
print(len(combined_docs.tolist()))
print(len(timestamps1))
print(len(topic_model.topics_))

# Check for NaT (Not a Time) values in your timestamps
print("Number of NaT values in timestamps:", sum(pd.isnull(timestamps1)))

# Now call the function with the adjusted data
topics_time = topic_model.topics_over_time(docs=combined_docs.tolist(),
                                           timestamps=timestamps1,
                                           global_tuning=True,
                                           evolution_tuning=True)

4179
4179
4179
Number of NaT values in timestamps: 0


11it [00:30,  2.77s/it]


In [23]:
topic_model.visualize_topics_over_time(topics_time)